In [1]:
%run WeightedQuickFind.ipynb
import random
from scipy.stats import norm

ModuleNotFoundError: No module named 'WeightedQuickFind'

In [2]:
class Percolation:

    # creates n-by-n grid, with all sites initially blocked
    def __init__(self, n):
        self.gridSize = n
        self._gridCells = WeigthedQuickUnion(self.gridSize**2 + 2)
        self._fullCells = WeigthedQuickUnion(self.gridSize**2 + 1)
        self.grid = [[1 for i in range(self.gridSize)] for j in range(self.gridSize)]
        self.__virtualBottom = self.gridSize**2 + 1
        self.__virtualTop = self.gridSize**2
        self.openSites = 0
    
    def isOnGrid(self, row, col):
        return ((row >= 0) & (col >= 0) & (row < self.gridSize) & (col < self.gridSize))
    

    # opens the site (row, col) if it is not open already
    def openSite(self, row, col):
        if(self.isOnGrid(row, col)):
            if self.isOpen(row, col):
                return
            else:
                self.grid[row][col] = 0
                self.openSites += 1
                flatIndex = self.flattenGrid(row, col) - 1
                if row == 1:
                    self._gridCells.union(self.__virtualTop, flatIndex)
                    self._fullCells.union(self.__virtualTop, flatIndex) 
                
                if row == self.gridSize:
                    self._gridCells.union(self.__virtualBottom, flatIndex)
                
                if self.isOnGrid(row, col - 1) & self.isOpen(row, col - 1):
                    self._gridCells.union(flatIndex, self.flattenGrid(row, col - 1) - 1)
                    self._fullCells.union(flatIndex, self.flattenGrid(row, col - 1) - 1)

                if self.isOnGrid(row, col + 1) & self.isOpen(row, col + 1):
                    self._gridCells.union(flatIndex, self.flattenGrid(row, col + 1) - 1)
                    self._fullCells.union(flatIndex, self.flattenGrid(row, col + 1) - 1)
    

                if self.isOnGrid(row - 1, col) & self.isOpen(row - 1, col):
                    self._gridCells.union(flatIndex, self.flattenGrid(row - 1, col) - 1)
                    self._fullCells.union(flatIndex, self.flattenGrid(row - 1, col) - 1)

                if self.isOnGrid(row + 1, col) & self.isOpen(row + 1, col):
                    self._gridCells.union(flatIndex, self.flattenGrid(row + 1, col) - 1)
                    self._fullCells.union(flatIndex, self.flattenGrid(row + 1, col) - 1)
                
                return
        else:
            print('Invalid entry')
            return

    # is the site (row, col) open?
    def isOpen(self, row, col):
        if(self.isOnGrid(row, col)):
            return(self.grid[row][col] == 0)
        else:
            return(False)
        
        
    def flattenGrid(self, row, col):
        return(self.gridSize * (row) + col)

    # is the site (row, col) full?
    def isFull(self, row, col):
        self.validateSite(row, col)
        return self._fullCells.connected(self.__virtualTop, self.flattenGrid(row, col) - 1);
    
    # returns the number of open sites
    def numberOfOpenSites(self):
        return(self.openSites)
    
    # does the system percolate?
    def percolates(self):
        return self._gridCells.connected(self.__virtualTop, self.__virtualBottom)
    
    def runTests(self):
        for row in range(1, self.gridSize + 1):
            for col in range(1, self.gridSize + 1): 
                if (self.isOpen(row, col)):
                    print("Row: {} Col: {} is Open".format(row, col))
                else:
                    print("Row: {} Col: {} is not Open".format(row, col))
                
                if (self.isFull(row, col)):
                    print("Row: {} Col: {} is Full".format(row, col))
                else:
                    print("Row: {} Col: {} is not Full".format(row, col))
                
            
        print("Sites Open: {}".format(self.numberOfOpenSites()))
        if (self.percolates()):
            print("Percolates /n")
        else:
            print("Does not Percolate /n")

In [3]:
class PercolationStats:
    
    def __init__(self, n, T):

        self.experiments = T
        self.fractions = []
        for i in range(T):
            self.__percolation = Percolation(n)
            self.__randrange = list(range(self.__percolation.flattenGrid(n-1, n-1)))
            random.shuffle(self.__randrange)
            while (not self.__percolation.percolates()):
                self.__r = self.__randrange.pop(0)
                self.__i = self.__r//n
                self.__j = self.__r%n
                if ~self.__percolation.isOpen(self.__i, self.__j):
                    self.__percolation.openSite(self.__i, self.__j)
                
            self.fractions.append(self.__percolation.openSites/n**2)
    
    def mean(self):
        return(sum(self.fractions)/len(self.fractions))
    
    def stddev(self):
        mu = self.mean()
        std = sum([(x - self.__mu)**2 for x in self.fractions])/(len(self.fractions)-1)
        return(std)
    
    def confidence_intervals(self, percent):
        sigma = norm.ppf(1 - (1-percent)/2)
        mu = self.mean()
        std = self.stddev()
        return(mu - sigma*std, mu + sigma*std)

In [4]:
def runOnInput(filename):

    with open(filename, 'r') as file:
        input_lines = [line.strip() for line in file]

    percolation = Percolation(int(input_lines[0]))
    for i in range(1, len(input_lines)):
        coordinates = input_lines[i].strip().split()
        r, c = int(coordinates[0]), int(coordinates[1])
        percolation.openSite(r, c)

In [ ]:
def runAll():
    